## 8.7 通过时间反向传播
到目前为止，我们已经反复提到像“梯度爆炸”、“梯度消失”，以及需要对循环神经网络“分离梯度”。例如，在8.5节中，我们在序列上调用了`detach`函数。为了能够快速构建模型并了解其工作原理，上面所说的这些概念都没有得到充分的解释。在本节中，我们将更深入地探讨序列模型反向传播的细节，以及相关的数学原理。

当我们首次实现循环神经网络时，遇到了梯度爆炸的问题。如果你做了练习题，就会发现梯度截断对于确保模型收敛至关重要。为了更好地理解此问题，本节将回顾序列模型梯度的计算方式，它的工作原理没有什么新概念，毕竟我们使用的仍然是链式法则来计算梯度。

我们在4.7节中描述了多层感知机中的前向与反向传播及相关的计算图。循环神经网络中的前向传播相对简单。通过时间反向传播(backpropagation through time, BPTT)实际上是循环神经网络中反向传播技术的一个特定应用。它要求我们将循环神经网络的计算图一次展开一个时间步，以获得模型变量和参数之间的依赖关系。然后，基于链式法则，应用反向传播计算和存储梯度。由于序列可能相当长，因此依赖关系也可能相当长。例如，某个1000个字符的序列，其第一个词元可能会最后位置的词元产生重大影响。这在计算上是不可行的，并且还需要超过1000个矩阵的乘积才能得到非常难以捉摸的梯度。这个过程充满了计算与统计的不确定性。在下文中，我们将阐明会发生什么以及如何在实践中解决他们。

### 8.7.1 循环神经网络的梯度分析
我们从一个描述循环神经网络工作原理的简化模型开始，此模型忽略了隐状态的特效及其更新方式的细节。这里的数学表示没有像过去那样明确地区分标量、向量和矩阵，这些细节并不重要。

在这个简化模型中，我们将时间步t的隐状态表示为$h_t$，输入表示为$x_t$，输出表示为$o_t$。回想一下我们在8.4.2节中的讨论，输入和隐状态可以拼接后与隐藏层的一个权重变量相乘。因此，我们分别使用$w_h$和$w_o$来表示隐藏层和输出层的权重。每个时间步的隐状态和输出可以写为：
$$
h_t = f(x_t, h_{t-1}, w_h)\\
o_t = g(h_t, w_o)
$$

其中，f和g分别是隐藏层和输出层的变换。因此，我们有一个链${..., (x_{t-1}, h_{t-1}, o_{t-1}), (x_t, h_t, o_t), ...}$，它们通过循环计算彼此依赖。前向传播相当简单，一次一个时间步的遍历三元组$(x_t, h_t, o_t)$，然后通过一个目标函数在所有T个时间步内评估输出$o_t$和对应的标签$y_t$之间的差异：
$$
L(x_1, ..., x_T, y_1, ..., y_T, w_h, w_o) = \frac{1}{T} \sum_{t=1}^T l(y_t, o_t)
$$

对于反向传播，问题则有些棘手，特别是当我们计算目标函数L关于参数$w_h$的梯度时。具体来说，按照链式法则：
$$
\begin{align*}
\frac {\partial L}{\partial w_h} & = \frac{1}{T} \sum_{t=1}^{T} \frac{\partial l(y_t, o_t)}{\partial w_h}\\
& = \frac{1}{T} \sum_{t=1}^{T} \frac{\partial l(y_t, o_t)}{\partial o_t} \frac{\partial g(h_t, w_o)}{\partial h_t} \frac{\partial h_t}{\partial w_h}
\end{align*}
$$

在(8.7.3)中乘积的第一项和第二项很容易计算，而第三项$$